# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [14]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [15]:
# check working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories 
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [16]:
# initiate an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line) 
            
#  get total number of rows 
print(len(full_data_rows_list))
#  check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [17]:
# check the number of rows 
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [18]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [19]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class': 'SimpleStrategy',
    'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [20]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [21]:
# Create song_info_by_session table with session_id and itemInSession as primary key
# artist_name, song_title and length of the song are queried based on the primary key

query = "CREATE TABLE IF NOT EXISTS song_info_by_session"
query = query + """(session_id int,
                    itemInSession int, 
                    artist_name text, 
                    song_title text, 
                    length float,
                    PRIMARY KEY(session_id, itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)     
                    

In [22]:
# Read data from file and insert data line by line to song_info_by_session table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO song_info_by_session(session_id,
                                           itemInSession,
                                           artist_name,
                                           song_title,
                                           length)"""
    
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), 
                                int(line[3]),
                                line[0], 
                                line[9], 
                                float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [23]:
# Verify the data was entered into the table

query = "SELECT artist_name, song_title, length FROM song_info_by_session WHERE session_id=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Returned query is based on artist_name, song_title, length of the song columns
for row in rows:
    print(row.artist_name, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [25]:
# Create song_info_by_id table with session_id and user_id as the partition key and itemInSession as clustering key
# artist_name, song_title, first_name, last_name, length, session_id, itemInSession, user_id are queried based on the primary key


query = "CREATE TABLE IF NOT EXISTS song_info_by_id"
query = query + """(session_id int,
                    user_id int,
                    itemInSession int, 
                    artist_name text, 
                    song_title text, 
                    first_name text,
                    last_name text,
                    length float, 
                    PRIMARY KEY((session_id,user_id), itemInSession))"""

try:
    session.execute(query)
except Exception as e:
    print(e)   
    
# Read data from file and insert data line by line to song_info_by_session table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO song_info_by_id(session_id,
                                                user_id, 
                                                itemInSession,
                                                artist_name,                  
                                                song_title,
                                                first_name,
                                                last_name,  
                                                length)"""
    
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),
                                int(line[10]),
                                int(line[3]),
                                line[0], 
                                line[9], 
                                line[1],
                                line[4],
                                float(line[5])))


##  Verify the data was entered into the table
query = "SELECT artist_name,song_title, first_name, last_name FROM song_info_by_id WHERE session_id=182 and user_id=10 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## Returned query is based on artist_name, song_title, the first_name, and last_name of user columns    
for row in rows:
    print(row.artist_name, 
          row.song_title, 
          row.first_name, 
          row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [26]:
# Create song_info_by_title table with song_title and user_id as the primary key to uniquely identify user
# User's first_name, last_name are queried based on the primary key

query = "CREATE TABLE IF NOT EXISTS song_info_by_title"
query = query + """(song_title text, 
                    user_id int,
                    first_name text,
                    last_name text,
                    PRIMARY KEY(song_title, user_id))"""
try:
    session.execute(query)
except Exception as e:
    print(e)   

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader: 
        query = """INSERT INTO song_info_by_title(song_title,
                                        user_id,
                                        first_name,
                                        last_name)"""
    
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], 
                                int(line[10]),
                                line[1],
                                line[4]))


query = "SELECT first_name, last_name, user_id FROM song_info_by_title WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Returned query is based on the first_name, last_name and user_id of the user 
for row in rows:
    print(row.first_name, 
          row.last_name,
         row.user_id)
                    

Jacqueline Lynch 29
Tegan Levine 80
Sara Johnson 95


### Drop the tables before closing out the sessions

In [27]:
## TO-DO: Drop the table before closing out the sessions
query = "drop table song_info_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_info_by_id"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    
query = "drop table song_info_by_title"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()